# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = []

for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)     
        for line in csvreader:
            full_data_rows_list.append(line)
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Creating tables and inserting data

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
                    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Creating queries to ask the following three questions of the data

### 1. Retrieve artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Here the Primary Key has two fields: item_in_session is the partition key, and session_id is the clustering key. Partitioning is done by item_in_session and within that partition, rows are ordered by the session_id.

In [8]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + "(artist text, item_in_session int, \
length float, session_id int, song text, \
PRIMARY KEY (item_in_session, session_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (artist, item_in_session, length, session_id, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))

#### Where item_in_session = 4 and session_id = 338, the artist is Faithless, the song is Music Matters (Mark Knight Dub), and its lenght is 495.307(about 8 minutes).

In [10]:
query = "select artist, song, length from song_playlist_session where item_in_session = 4 AND session_id = 338"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

data = []
for row in rows:
    data.append([row.artist, row.song, row.length])

df = pd.DataFrame(data, columns=['artist', 'song', 'lenght'])
df.style.hide_index()

artist,song,lenght
Faithless,Music Matters (Mark Knight Dub),495.307


### 2. Retrieve only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


Here the Primary Key has three fields: (session_id, user_id) is the partition key, and item_in_session is the clustering key. Partitioning is done by session_id and user_id, and within that partition, rows are ordered by the item_in_session.


In [11]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_user_session"
query = query + "(artist text, first_name text, item_in_session int, \
last_name text, session_id int, song text, user_id int, \
PRIMARY KEY ((session_id, user_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_playlist_user_session (artist, first_name, item_in_session, last_name, \
         session_id, song, user_id)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[1], int(line[3]), line[4], int(line[8]), line[9], int(line[10])))

#### Where session_id = 182 and user_id = 10, four records are retrieved. The artist are Down To The Bone, Three Drives, Sebastien Tellier and Lonnie Gordon, in that order.

In [14]:
query = "select artist, song, first_name, last_name from song_playlist_user_session where session_id = 182 and user_id = 10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

data = []
for row in rows:
    data.append([row.artist, row.song, row.first_name, row.last_name])

df = pd.DataFrame(data, columns=['artist', 'song', 'first_name', 'last_name'])
df.style.hide_index()

artist,song,first_name,last_name
Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
Three Drives,Greece 2000,Sylvie,Cruz
Sebastien Tellier,Kilometer,Sylvie,Cruz
Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio Edit),Sylvie,Cruz


## 3. Retrieve every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'


Here the Primary Key has two fields: song is the partition key, and user_id is the clustering key. Partitioning is done by song, and within that partition, rows are ordered by the user_id.

In [15]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_user_name"
query = query + "(first_name text, last_name text, song text, user_id int, \
PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_playlist_user_name (first_name, last_name, song, user_id)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1],line[4], line[9], int(line[10])))

#### Three rows are returned by the query where song = 'All Hands Against His Own'. Users names are: Jacqueline Lynch, Tegan Levine, Sara Johnson.

In [18]:
query = "select first_name, last_name from song_playlist_user_name where song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

data = []
for row in rows:
    data.append([row.first_name, row.last_name])

df = pd.DataFrame(data, columns=['first_name', 'last_name'])
df.style.hide_index()

first_name,last_name
Jacqueline,Lynch
Tegan,Levine
Sara,Johnson


### Drop the tables before closing out the sessions

In [50]:
query = "DROP TABLE IF EXISTS song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS song_playlist_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS song_playlist_user_name"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [51]:
session.shutdown()
cluster.shutdown()